#Sahil's space

## Imports and data 

In [ ]:
!pip install segmentation-models-pytorch

     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 245kB 33.1MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=fb56d408d73e0d1f078166f5197d7119d247fecfc1c097d25c9bb9c1973ce500
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60963 sha256=c99027260db500e2f46930ab6704bada8e411e9a5c31dace60b874c4de0a7e0c
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
################# IMPORT PACKAGES #############
%matplotlib inline
import os
import sys
import cv2
import random
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import image
from tqdm import tqdm
import math
from PIL import Image
from google.colab.patches import cv2_imshow
import sklearn
from sklearn import model_selection
from sklearn import metrics

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torch.nn as nn
from collections import defaultdict
import torchvision
from torchvision import transforms
import torch.nn.functional as F
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter

import segmentation_models_pytorch 

import albumentations as A
#from albumentations.pytorch import ToTensorV2
from albumentations.pytorch.transforms import ToTensor

##To get the data:<br>
Track 1: https://drive.google.com/drive/folders/1sZZMJkbqJNbHgebKvHzcXYZHJd6ss4tH <br>
Track 2: https://drive.google.com/drive/folders/1g1r419bWBe4GEF-7si5DqWCjxiC8ErnY <br>
Open and add shortcut to MyDrive

In [ ]:
################ MOUNTING DRIVE ##################
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/512_Images/512_Images.zip

Archive:  /content/drive/MyDrive/512_Images/512_Images.zip
   creating: 512_Images/
   creating: 512_Images/Test/
   creating: 512_Images/Test/image/
  inflating: 512_Images/Test/image/10823.jpg  
  inflating: 512_Images/Test/image/7611.jpg  
  inflating: 512_Images/Test/image/8007.jpg  
  inflating: 512_Images/Test/image/7652.jpg  
  inflating: 512_Images/Test/image/7730.jpg  
  inflating: 512_Images/Test/image/6671.jpg  
  inflating: 512_Images/Test/image/8391.jpg  
  inflating: 512_Images/Test/image/9886.jpg  
  inflating: 512_Images/Test/image/7874.jpg  
  inflating: 512_Images/Test/image/8544.jpg  
  inflating: 512_Images/Test/image/7029.jpg  
  inflating: 512_Images/Test/image/8376.jpg  
  inflating: 512_Images/Test/image/7031.jpg  
  inflating: 512_Images/Test/image/8494.jpg  
  inflating: 512_Images/Test/image/6353.jpg  
  inflating: 512_Images/Test/image/9113.jpg  
  inflating: 512_Images/Test/image/8542.jpg  
  inflating: 512_Images/Test/image/8154.jpg  
  inflating: 512_Imag

In [ ]:
%cd 512_Images

/content/512_Images


In [ ]:
!ls Train/Labeled

Flooded  Non-Flooded


#Saving Masks

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
!ls Train/Labeled
print(len(os.listdir('/content/512_Images/Train/Labeled/Flooded/image')))
print(len(os.listdir('/content/512_Images/Train/Labeled/Non-Flooded/image')))
print(len(os.listdir('/content/512_Images/Train/Labeled/Flooded/mask')))
print(len(os.listdir('/content/512_Images/Train/Labeled/Non-Flooded/mask')))

ls: cannot access 'Train/Labeled': No such file or directory
51
347
51
347


In [ ]:
imgs = [os.path.join("Train/Labeled/Flooded/image", img_n) for img_n in os.listdir('/content/512_Images/Train/Labeled/Flooded/image')]
imgs += [os.path.join("Train/Labeled/Non-Flooded/image", img_n) for img_n in os.listdir('/content/512_Images/Train/Labeled/Non-Flooded/image')]
imgs = sorted(imgs)
print(len(imgs), imgs[0])

398 Train/Labeled/Flooded/image/10165.jpg


In [ ]:
imgs[0]

'Train/Labeled/Flooded/image/10165.jpg'

In [ ]:
347 + 51

398

In [ ]:
DIR_PATH='/content/512_Images'
MODEL_PATH_UNET='/content/drive/MyDrive/models/_Unet_anki_final40.pt'
MODEL_PATH_PSP= '/content/drive/MyDrive/models/final-PSPNet_2-resnet101-ep_101-0.001-24.pt'
MODEL_PATH_DL='/content/drive/MyDrive/models/abhi-pretrained_preproc_1-deeplabv3+-ep_79-0.001-8.pt'
MODEL_PATH_DLPS= '/content/drive/MyDrive/models/abhi_sudo-pretrained_preproc_1-deeplabv3+-ep_20-0.001-8.pt'

In [ ]:
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
def reverse_transform_mask(inp):
    inp=inp.transpose((1, 2, 0))
    t_mask=np.argmax(inp,axis=2).astype('float32') 
    t_mask=cv2.resize(t_mask, dsize=(4000, 3000))
    return t_mask

In [ ]:
imgs[0]

'10165.jpg'

In [ ]:
class ValDataset:
    def __init__(self, dir_path, imgs, val_trans, img_dim):
        self.dir_path=dir_path
        self.imgs = imgs
        self.img_dim = img_dim
        self.val_trans=val_trans

    def __len__(self):
      return len(self.imgs)

    def __getitem__(self, index):
      # image = cv2.resize(cv2.imread(os.path.join(self.dir_path,self.imgs[index])), dsize=(self.img_dim, self.img_dim))
      image = cv2.imread(os.path.join(self.dir_path,self.imgs[index]))
      if self.val_trans:
        image=self.val_trans(image)
      return image, index

In [ ]:
LOAD_SIZE= 10
val_set = ValDataset(DIR_PATH,imgs, val_transform , img_dim=IMG_DIM)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=LOAD_SIZE, shuffle=False)

In [ ]:
# model_name = "UNET" # DV3S PSP DV3 UNET
# SAVE_PATH=f'/content/drive/MyDrive/512_Images/inferences-{model_name}'
# os.makedirs(SAVE_PATH, exist_ok=True)
# if model_name == "DV3S":
#   model = segmentation_models_pytorch.DeepLabV3Plus(encoder_name='efficientnet-b3', encoder_depth=5, 
#                                                   decoder_channels=256,classes=10)
#   model.load_state_dict(torch.load(MODEL_PATH_DLPS))
# elif model_name == "PSP":
#   model = segmentation_models_pytorch.PSPNet(encoder_name='resnet101', encoder_weights='imagenet', encoder_depth=3, 
#                                                  psp_out_channels=512, psp_use_batchnorm=True, psp_dropout=0.2, in_channels=3, 
#                                                  classes=10, activation=None, upsampling=8, aux_params=None)
#   model.load_state_dict(torch.load(MODEL_PATH_PSP))
# elif model_name == "DV3":
#   model = segmentation_models_pytorch.DeepLabV3Plus(encoder_name='efficientnet-b3', encoder_depth=5, 
#                                                   decoder_channels=256,classes=10)
#   model.load_state_dict(torch.load(MODEL_PATH_DL))
# elif model_name == "UNET":
#   model = segmentation_models_pytorch.Unet(encoder_name='resnet34', encoder_depth=5, 
#                                          decoder_channels=(256, 128, 64, 32, 16), classes=10)
#   model.load_state_dict(torch.load(MODEL_PATH_UNET))

# model = model.to(device)
# model.eval()
# for inputs, index in tqdm(val_dataloader):
#   inputs = inputs.to(device)
#   preds = model(inputs)
#   preds = F.sigmoid(preds)
#   preds = preds.data.cpu().numpy()
#   for i in range(min(LOAD_SIZE, len(index))):
#     f_mask=reverse_transform_mask(preds[i])
#     cv2.imwrite(os.path.join(SAVE_PATH,str(imgs[index[i]].split("/")[-1].replace("jpg", "png"))),f_mask)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
len(os.listdir(SAVE_PATH)) # 398

NameError: ignored

In [ ]:
masks = [os.path.join("/content/512_Images/Train/Labeled/Flooded/mask", img_n) for img_n in os.listdir('/content/512_Images/Train/Labeled/Flooded/mask')]
masks += [os.path.join("/content/512_Images/Train/Labeled/Non-Flooded/mask", img_n) for img_n in os.listdir('/content/512_Images/Train/Labeled/Non-Flooded/mask')]
masks = sorted(masks)
print(len(masks), masks[0])

398 /content/512_Images/Train/Labeled/Flooded/mask/10165_lab.png


In [ ]:
# models = ["DV3S", "PSP", "DV3", "UNET",]
imgs = sorted([os.path.join('/content/512_Images', i) for i in imgs])

unet_m = sorted(os.listdir(f'/content/drive/MyDrive/512_Images/inferences-UNET'))
unet_m = sorted([os.path.join('/content/drive/MyDrive/512_Images/inferences-UNET', i) for i in unet_m])

psp_m = sorted(os.listdir(f'/content/drive/MyDrive/512_Images/inferences-PSP'))
psp_m = sorted([os.path.join('/content/drive/MyDrive/512_Images/inferences-PSP', i) for i in psp_m])

dv3_m = sorted(os.listdir(f'/content/drive/MyDrive/512_Images/inferences-DV3'))
dv3_m = sorted([os.path.join('/content/drive/MyDrive/512_Images/inferences-DV3', i) for i in dv3_m])

dv3s_m = sorted(os.listdir(f'/content/drive/MyDrive/512_Images/inferences-DV3S'))
dv3s_m = sorted([os.path.join('/content/drive/MyDrive/512_Images/inferences-DV3S', i) for i in dv3s_m])

grn_t = sorted(masks)

print(len(imgs), imgs[0])
print(len(unet_m), unet_m[0])
print(len(psp_m), psp_m[0])
print(len(dv3_m), dv3_m[0])
print(len(dv3s_m), dv3s_m[0])
print(len(grn_t), grn_t[0])

398 /content/512_Images/Train/Labeled/Flooded/image/10165.jpg
398 /content/drive/MyDrive/512_Images/inferences-UNET/10165.png
398 /content/drive/MyDrive/512_Images/inferences-PSP/10165.png
398 /content/drive/MyDrive/512_Images/inferences-DV3/10165.png
398 /content/drive/MyDrive/512_Images/inferences-DV3S/10165.png
398 /content/512_Images/Train/Labeled/Flooded/mask/10165_lab.png


In [ ]:
count = 0
for im, un, ps, dv, ds, g in zip(imgs, unet_m, psp_m, dv3_m, dv3s_m, grn_t):
  im = im.split("/")[-1].split(".")[0]
  un = f"/content/drive/MyDrive/512_Images/inferences-UNET/{im}.png"
  un = un.split("/")[-1].split(".")[0]
  ps = f"/content/drive/MyDrive/512_Images/inferences-PSP/{im}.png"
  ps = ps.split("/")[-1].split(".")[0]
  dv = f"/content/drive/MyDrive/512_Images/inferences-DV3/{im}.png"
  dv = dv.split("/")[-1].split(".")[0]
  ds = f"/content/drive/MyDrive/512_Images/inferences-DV3S/{im}.png"
  ds = ds.split("/")[-1].split(".")[0]
  g = f"/content/512_Images/Train/Labeled/Flooded/mask/{im}_lab.png"
  g = g.split("/")[-1].split(".")[0].replace("_lab", "")
  if not (im == un == ps == dv == ds == g):
    # print(im)
    count += 1 
print(count)

0


In [ ]:
num_imgs = 2

In [ ]:
len(im_paths)

18

In [ ]:

math.ceil(398/3)
print(len(imgs))

398


In [ ]:
threes = 0
twos = 0
for i in range(math.ceil(len(imgs) / 3)):
  num_imgs = min(3, len(imgs[i*3:i*3+3]))
  if num_imgs == 2:
    twos += 1
  elif num_imgs == 3:
    threes += 1
print(twos, threes)

1 132


In [ ]:
# total images = 398
# 3 images per plt
os.makedirs('/content/drive/MyDrive/512_Images/plt_inference/', exist_ok=True)
for i in tqdm(range(math.ceil(len(imgs) / 3))):
  num_imgs = min(3, len(imgs[i*3:i*3+3]))
  ims = imgs[i*3:i*3+3]
  uns = []
  pss = []
  dvs = []
  dss = []
  gs = []
  for im1 in ims:
    im = im1.split("/")[-1].split(".")[0]
    uns.append(f"/content/drive/MyDrive/512_Images/inferences-UNET/{im}.png")
    pss.append(f"/content/drive/MyDrive/512_Images/inferences-PSP/{im}.png")
    dvs.append(f"/content/drive/MyDrive/512_Images/inferences-DV3/{im}.png")
    dss.append(f"/content/drive/MyDrive/512_Images/inferences-DV3S/{im}.png")
    gs.append(im1.replace("/image/", "/mask/").replace(".jpg", "_lab.png"))

  im_paths = []
  for n in range(num_imgs):
    im_paths.extend([ims[n], uns[n], pss[n], dvs[n], dss[n], gs[n]])
  count = 0
  fig = plt.figure(figsize=(14, 5))
  for n in range(num_imgs):
      for num, image_title in enumerate(["(a) Image", "(b) UNet", "(c) PSPNet", "(d) DeepLabV3+", "(e) DeepLabV3+ Pseudo", "(f) Ground Truth"]):
          fig.add_subplot(3, 6, count+1)
          if image_title == "(f) Ground Truth":
              plt.imshow(cv2.resize(image.imread(im_paths[count])[:, :, 0], (4000, 3000)))
          elif image_title == "(a) Image":
            plt.imshow(cv2.resize(image.imread(im_paths[count]), (4000, 3000)))
          else:
              plt.imshow(image.imread(im_paths[count]))
          plt.axis("off")
          if count < 6:
              plt.title(image_title)
          count += 1
  plt.savefig(f'/content/drive/MyDrive/512_Images/plt_inference/{i}.png')
  plt.clf()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

...

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

<Figure size 1008x360 with 0 Axes>

In [ ]:
imgs[0:3]

['/content/512_Images/Train/Labeled/Flooded/image/10165.jpg',
 '/content/512_Images/Train/Labeled/Flooded/image/10166.jpg',
 '/content/512_Images/Train/Labeled/Flooded/image/6279.jpg']

In [ ]:
with open("img_paths.txt", "w") as f:
  for im in imgs:
    f.write(im.replace("/content/512_Images/", "") + "\n")
f.close()

In [ ]:
!zip -r plt_inference.zip /content/drive/MyDrive/512_Images/plt_inference/

  adding: content/drive/MyDrive/512_Images/plt_inference/ (stored 0%)
  adding: content/drive/MyDrive/512_Images/plt_inference/0.png (deflated 2%)
  adding: content/drive/MyDrive/512_Images/plt_inference/1.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/2.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/3.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/4.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/5.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/6.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/7.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/8.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/9.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/10.png (deflated 1%)
  adding: content/drive/MyDrive/512_Images/plt_inference/11.png (deflated 1%)
  ad